In [79]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import json
import pandas as pd


In [80]:
job_title = 'Machine learning python'
vacancies_result = []


In [82]:
hh_main_link = 'https://hh.ru'
hh_link_folder = '/search/vacancy'
hh_url = hh_main_link + hh_link_folder

hh_params = {'area': 1,
          'st': 'searchVacancy',
          'fromSearch': 'true',
          'text': job_title}

headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15'}

hh_next_page_button_exists = True

while hh_next_page_button_exists:

    hh_response = requests.get(hh_url, headers=headers, params=hh_params)
    hh_soup = bs(hh_response.text,'lxml')

    hh_vacancies_block = hh_soup.find('div', {'class': 'vacancy-serp'})
    hh_vacancies_list = hh_vacancies_block.find_all('div', {'data-qa': 'vacancy-serp__vacancy', 'class': 'vacancy-serp-item'})

    for vacancy in hh_vacancies_list:
        hh_vacancy_data = {}
        hh_vacancy_data['name'] = vacancy.find('a').getText()
        hh_vacancy_data['vacancy_link'] = vacancy.find('a')['href']
        hh_vacancy_data['employer_source'] = hh_main_link + vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'})['href']
        hh_vacancy_data['employer'] = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info'}).getText()
        salary = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        max = re.search(r"(?<=до ).*?(?= )|(?<=-)\d*\s\d*", salary)
        min = re.search(r"(?<=от ).*?(?= )|.*?(?=-)", salary)
        curr = re.search(r".{4}$", salary)
        if max:
            hh_vacancy_data['salary_max'] = max.group(0).replace('\xa0000', '000')
        else:
            hh_vacancy_data['salary_max'] = None
        if min:
            hh_vacancy_data['salary_min'] = min.group(0).replace('\xa0000', '000')
        else:
            hh_vacancy_data['salary_min'] = None
        if curr:
            hh_vacancy_data['salary_currency'] = curr.group(0)
        else:
            hh_vacancy_data['salary_currency'] = None
        vacancies_result.append(hh_vacancy_data)

    hh_next_page_block = hh_soup.find('a', {'class': 'HH-Pager-Controls-Next'})
    if hh_next_page_block:
        hh_next_page_button_exists = True
        hh_next_page_url = hh_next_page_block['href']
        hh_url = hh_main_link + hh_next_page_url
    else:
        hh_next_page_button_exists = False

pprint(vacancies_result)

[{'employer': ' Smart Group\xa0',
  'employer_source': 'https://hh.ru/employer/683450',
  'name': 'ИТ-архитектор',
  'salary_currency': 'руб.',
  'salary_max': '370000',
  'salary_min': '250000',
  'vacancy_link': 'https://hh.ru/vacancy/37648464?query=Machine%20learning%20python'},
 {'employer': ' FunCorp\xa0',
  'employer_source': 'https://hh.ru/employer/1212374',
  'name': 'Senior Python Backend Developer (Machine Learning)',
  'salary_currency': 'руб.',
  'salary_max': None,
  'salary_min': '250000',
  'vacancy_link': 'https://hh.ru/vacancy/37369634?query=Machine%20learning%20python'},
 {'employer': 'ООО Claire&Clarté\xa0',
  'employer_source': 'https://hh.ru/employer/2370983',
  'name': 'Junior Machine Learning Engineer',
  'salary_currency': 'руб.',
  'salary_max': '200',
  'salary_min': None,
  'vacancy_link': 'https://hh.ru/vacancy/37711158?query=Machine%20learning%20python'},
 {'employer': ' Gradient\xa0',
  'employer_source': 'https://hh.ru/employer/4111056',
  'name': 'Senior

In [209]:
sj_main_link = 'https://www.superjob.ru'
sj_link_folder = '/vacancy/search'
sj_url = sj_main_link + sj_link_folder

sj_params = {'keywords': job_title,
             'page': 0}

headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15'}

sj_next_page_button_exists = True

while sj_next_page_button_exists:

    sj_response = requests.get(sj_url, headers=headers, params=sj_params)
    sj_soup = bs(sj_response.text,'lxml')

    sj_vacancies_list = sj_soup.find_all('div', {'class': 'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})

    for vacancy in sj_vacancies_list:
        sj_vacancy_data = {}
        sj_vacancy_data['name'] = vacancy.find('div', {'class': '_3mfro'}).getText()
        sj_vacancy_data['vacancy_link'] = sj_main_link + vacancy.find('a', {'class': '_1UJAN'})['href']
        sj_vacancy_data['employer_source'] = sj_main_link + vacancy.find('a', {'class': '_25-u7'})['href']
        sj_vacancy_data['employer'] = vacancy.find('a', {'class': '_25-u7'}).getText()
        salary = vacancy.find('span', {'class': '_3mfro'}).getText()
        salary_split = re.split('[\xa0 -]', salary)
        sj_vacancy_data['salary_currency'] = salary_split[-1]
        if salary == 'По договорённости':
            sj_vacancy_data['salary_max'] = None
            sj_vacancy_data['salary_min'] = None
            sj_vacancy_data['salary_currency'] = None
        if len(salary_split) == 4 and salary_split[0] == 'до':
            sj_vacancy_data['salary_max'] = int(salary_split[1] + salary_split[2])
        elif len(salary_split) == 4 and salary_split[0] == 'от':
            sj_vacancy_data['salary_min'] = int(salary_split[1] + salary_split[2])
        elif len(salary_split) > 4:
            sj_vacancy_data['salary_max'] = int(salary_split[3] + salary_split[4])
            sj_vacancy_data['salary_min'] = int(salary_split[0] + salary_split[1])

        vacancies_result.append(sj_vacancy_data)

    sj_next_page_block = sj_soup.find('a', {'rel': 'next', 'class': 'f-test-link-Dalshe'})

    if sj_next_page_block:
        sj_next_page_button_exists = True
        sj_params['page'] += 1
    else:
        sj_next_page_button_exists = False

pprint(vacancies_result)

[{'employer': ' Smart Group\xa0',
  'employer_source': 'https://hh.ru/employer/683450',
  'name': 'ИТ-архитектор',
  'salary_currency': 'руб.',
  'salary_max': '370000',
  'salary_min': '250000',
  'vacancy_link': 'https://hh.ru/vacancy/37648464?query=Machine%20learning%20python'},
 {'employer': ' FunCorp\xa0',
  'employer_source': 'https://hh.ru/employer/1212374',
  'name': 'Senior Python Backend Developer (Machine Learning)',
  'salary_currency': 'руб.',
  'salary_max': None,
  'salary_min': '250000',
  'vacancy_link': 'https://hh.ru/vacancy/37369634?query=Machine%20learning%20python'},
 {'employer': 'ООО Claire&Clarté\xa0',
  'employer_source': 'https://hh.ru/employer/2370983',
  'name': 'Junior Machine Learning Engineer',
  'salary_currency': 'руб.',
  'salary_max': '200',
  'salary_min': None,
  'vacancy_link': 'https://hh.ru/vacancy/37711158?query=Machine%20learning%20python'},
 {'employer': ' Gradient\xa0',
  'employer_source': 'https://hh.ru/employer/4111056',
  'name': 'Senior

In [210]:
df = pd.DataFrame(vacancies_result)
df

,name,vacancy_link,employer_source,employer,salary_max,salary_min,salary_currency
0,ИТ-архитектор,https://hh.ru/vacancy/37648464?query=Machine%2...,https://hh.ru/employer/683450,Smart Group,370000,250000,руб.
1,Senior Python Backend Developer (Machine Learn...,https://hh.ru/vacancy/37369634?query=Machine%2...,https://hh.ru/employer/1212374,FunCorp,None,250000,руб.
2,Junior Machine Learning Engineer,https://hh.ru/vacancy/37711158?query=Machine%2...,https://hh.ru/employer/2370983,ООО Claire&Clarté,200,None,руб.
3,Senior Data Scientist,https://hh.ru/vacancy/37935294?query=Machine%2...,https://hh.ru/employer/4111056,Gradient,None,300000,руб.
4,Data Scientist (Computer Vision),https://hh.ru/vacancy/38090271?query=Machine%2...,https://hh.ru/employer/4046921,Tevian (ООО Технологии видеоанализа),200000,90000,руб.
...,...,...,...,...,...,...,...
151,ИТ-архитектор,https://hh.ru/vacancy/37648464?query=Machine%2...,https://hh.ru/employer/683450,Smart Group,370000,250000,руб.
152,Аналитик данных (Data Analyst/Data Scientist),https://www.superjob.ru/vakansii/analitik-dann...,https://www.superjob.ru/clients/standartproekt...,СтандартПроект,None,None,None
153,"Data scientist (аналитик, разработчик данных)",https://www.superjob.ru/vakansii/data-scientis...,https://www.superjob.ru/clients/berezina-lebed...,Березина-Лебедева И. В.,None,None,None
154,Аналитик данных (Data Analyst/Data Scientist),https://www.superjob.ru/vakansii/analitik-dann...,https://www.superjob.ru/clients/standartproekt...,СтандартПроект,None,None,None


In [86]:
df.to_json('parcer_results_hh_sj.json', orient = 'records', lines = True)

In [211]:
import json
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['vacancies']
vacancies = db.vacancies


In [212]:
vac_dict = df.to_dict('records')
vac_dict

vacancies.insert_many(vac_dict)

In [213]:
for vac in vacancies.find({}):
    pprint(vac)

{'_id': ObjectId('5f1c074cd9fd536b27e732d8'),
 'employer': ' Smart Group\xa0',
 'employer_source': 'https://hh.ru/employer/683450',
 'name': 'ИТ-архитектор',
 'salary_currency': 'руб.',
 'salary_max': '370000',
 'salary_min': '250000',
 'vacancy_link': 'https://hh.ru/vacancy/37648464?query=Machine%20learning%20python'}
{'_id': ObjectId('5f1c074cd9fd536b27e732d9'),
 'employer': ' FunCorp\xa0',
 'employer_source': 'https://hh.ru/employer/1212374',
 'name': 'Senior Python Backend Developer (Machine Learning)',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': '250000',
 'vacancy_link': 'https://hh.ru/vacancy/37369634?query=Machine%20learning%20python'}
{'_id': ObjectId('5f1c074cd9fd536b27e732da'),
 'employer': 'ООО Claire&Clarté\xa0',
 'employer_source': 'https://hh.ru/employer/2370983',
 'name': 'Junior Machine Learning Engineer',
 'salary_currency': 'руб.',
 'salary_max': '200',
 'salary_min': None,
 'vacancy_link': 'https://hh.ru/vacancy/37711158?query=Machine%20learning%

In [223]:
def salary_threshold(threshold):
    for i in vacancies.find( { '$or': [{'salary_min': { '$gte': threshold}}, {'salary_max': { '$gte': threshold}}]}):
        pprint(i)
        
salary_threshold(0)
    

{'_id': ObjectId('5f1c1336d9fd536b27e735d8'),
 'employer': 'Березина-Лебедева И. В.',
 'employer_source': 'https://www.superjob.ru/clients/berezina-lebedeva-i-v-2817533.html',
 'name': 'Разработчик (full-stack)',
 'salary_currency': 'руб.',
 'salary_max': 160000,
 'salary_min': nan,
 'vacancy_link': 'https://www.superjob.ru/vakansii/razrabotchik-34191453.html'}


In [242]:
new = {
 'employer': 'Березина-Лебедева И. В.',
 'employer_source': 'https://www.superjob.ru/clients/berezina-lebedeva-i-v-2817533.html',
 'name': 'Разработчик (full-stack)',
 'salary_currency': 'руб.',
 'salary_max': 180000,
 'salary_min': 'nan',
 'vacancy_link': 'https://www.superjob.ru/vakansii/razrabotchik-34191453.html'}

for i in vacancies.find({}):
    vacancies.update_one({'vacancy_link':new['vacancy_link']},
                         {'$set':new},
                         upsert=True,
                         bypass_document_validation=True)
    
for i in vacancies.find():
    pprint(i)

{'_id': ObjectId('5f1c074cd9fd536b27e732d8'),
 'employer': ' Smart Group\xa0',
 'employer_source': 'https://hh.ru/employer/683450',
 'name': 'ИТ-архитектор',
 'salary_currency': 'руб.',
 'salary_max': '370000',
 'salary_min': '250000',
 'vacancy_link': 'https://hh.ru/vacancy/37648464?query=Machine%20learning%20python'}
{'_id': ObjectId('5f1c074cd9fd536b27e732d9'),
 'employer': ' FunCorp\xa0',
 'employer_source': 'https://hh.ru/employer/1212374',
 'name': 'Senior Python Backend Developer (Machine Learning)',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': '250000',
 'vacancy_link': 'https://hh.ru/vacancy/37369634?query=Machine%20learning%20python'}
{'_id': ObjectId('5f1c074cd9fd536b27e732da'),
 'employer': 'ООО Claire&Clarté\xa0',
 'employer_source': 'https://hh.ru/employer/2370983',
 'name': 'Junior Machine Learning Engineer',
 'salary_currency': 'руб.',
 'salary_max': '200',
 'salary_min': None,
 'vacancy_link': 'https://hh.ru/vacancy/37711158?query=Machine%20learning%